In [1]:
// Скачаем Akka
import $ivy.`com.typesafe.akka:akka-actor_2.11:2.5.3`

import $ivy.$                                        

# Akka и с чем его едят

В акторной модели – которая была изобретена в 1973 году Карлом Хьюиттом и др. — акторы представляют собой «фундаментальные единицы вычислений, реализующие обработку, хранение и коммуникацию». 

![](http://vignette2.wikia.nocookie.net/wikies/images/7/73/%D0%9C%D0%B8%D0%BD%D1%8C%D0%BE%D0%BD%D1%8B%D0%A4%D0%BE%D0%BD.jpg/revision/latest?cb=20150703140704&path-prefix=ru)

Если не вдаваться в подробности, то разработка на акторах исходит из философии, что все круг акторы. Так же как и ООП исходит из философии, что все круг объекты. Принципиальные отличия же состоят в том, что акторы выполняются параллельно. В то время как ООП код выполняется последовательно и для параллельного исполнения надо делать дополнительные и далеко не всегда простые действия. А так же акторы взаимодействуют между собой не через вызовы методов у объектов, как в ООП, а через отправку сообщений. В акторе есть очередь этих сообщений (mailbox). Сообщения обрабатываются строго по очереди.

Кроме того, необходимо отметить, что отправка сообщения актору и обработка этого сообщения актором — это две отдельных операции, которые, скорее всего, происходят в разных потоках. Разумеется, Akka обеспечивает необходимую синхронизацию, чтобы гарантировать, что любые изменения состояния будут видимы всем потокам. 


In [8]:
import akka.actor._

import akka.actor._

In [3]:
class PubSubMediator extends Actor {
  override def receive = Actor.emptyBehavior
}

defined class PubSubMediator

Метод receive возвращает так называемое исходное поведение актора. Это просто частично вычислимая функция, используемая Akka для обработки сообщений, отправляемых актору.

При создании акторной системы, Akka — на внутреннем уровне использующая множество так называемых «системных акторов» — создает три актора: это «корневой страж» (root guardian), расположенный в корне акторной иерархии, а также системный и пользовательский стражи. Пользовательский страж — зачастую именуемый просто «страж» — является родительским элементом для всех создаваемых нами акторов верхнего уровня (в данном контексте имеется в виду «наивысший уровень, к которому мы имеем доступ»).


In [4]:
// Создадим систему акторов
val system = ActorSystem("pub-sub-mediator-spec-system")

system: ActorSystem = akka://pub-sub-mediator-spec-system

А зачем мы вообще создаем ActorSystem? Почему бы просто не создавать акторы? Последнее невозможно, поскольку при непосредственном вызове конструктора актора система выбросит исключение. Вместо этого нам придется использовать фабричный метод, предоставляемый — вы угадали — ActorSystem для создания актора верхнего уровня:

In [5]:
system.actorOf(Props(new PubSubMediator), "pub-sub-mediator")

res4: ActorRef = Actor[akka://pub-sub-mediator-spec-system/user/pub-sub-mediator#-8146321]

In [6]:
object PubSubMediator {
 
  final val Name = "pub-sub-mediator"
 
  def props: Props = Props(new PubSubMediator)
}

defined object PubSubMediator

А что за такая штуковина Props? Это просто конфигурационный объект для актора. Он принимает конструктор как параметр, передаваемый по имени (то есть, лениво) и может содержать другую важную информацию – например, о маршрутизации или развертывании. 

In [10]:
// Если мы захотим создать актор с таким же именем
// Мы этого не сможем сделать
// Ибо у каждого актора должно быть уникальное имя
system.actorOf(PubSubMediator.props, PubSubMediator.Name)

: 